# Анализ результатов эксперимента от 26.09.25

In [1]:
%load_ext autoreload
%autoreload 2

## Описание эксперимента

Перебор коэффициентов по равномерной сетке. Setpoint = 1200. Сетка 7*7*7, не включая граничные точки.

## Результаты

In [1]:
from pathlib import Path

EXPERIMENT_NAME = "pid_grid_scan"
EXPERIMENT_DATE = "2025-09-26"
EXPERIMENT_TIME = "16-48-36"

PATH_TO_EXP_DIR = Path(f"../experiments/{EXPERIMENT_NAME}/{EXPERIMENT_DATE}/{EXPERIMENT_TIME}")
LOG_DIR = PATH_TO_EXP_DIR / "logs"

In [2]:
import pandas as pd
import re
from collections import defaultdict

def parse_pid_logfile(path: str) -> pd.DataFrame:
    """
    Читает лог PID-контроллера из файла и возвращает DataFrame.
    Для каждого step объединяет send и recv в одну строку.
    """
    pattern = re.compile(r"(\w+)=([0-9.+-eE]+)")
    steps = defaultdict(dict)

    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            parts = line.split(maxsplit=3)
            step = int(parts[0].split("=")[1])
            timestamp = float(parts[1].split("=")[1])
            direction = parts[2]  # send / recv

            steps[step][f"time_{direction}"] = timestamp

            if len(parts) > 3:
                for key, value in pattern.findall(parts[3]):
                    steps[step][key] = float(value)

    df = pd.DataFrame.from_dict(steps, orient="index").reset_index()
    df = df.rename(columns={"index": "step"})
    return df.sort_values("step").reset_index(drop=True)


In [ ]:
log_df = parse_pid_logfile(LOG_DIR / "log.txt")

SETPOINT = 1200
log_df["setpoint"] = SETPOINT

log_df["error"] = -abs(log_df["process_variable"] - SETPOINT)

In [ ]:
print(log_df.head())

In [ ]:
print(log_df.info())

In [ ]:
print(log_df.describe())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_df(ax, df, tags=None, title="Training curves"):
    for tag in tags:
        sns.lineplot(data=df, x="step", y=tag, ax=ax, label=tag)
    
    ax.set_title(title, fontsize=14)
    ax.set_xlabel("Step")
    ax.set_ylabel("Value")
    ax.legend(title="Metric")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_df(ax, log_df, tags=["process_variable", "control_output"], title="Observation curves")
plt.tight_layout()
plt.savefig(LOG_DIR / "logs.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_df(ax, log_df, tags=["process_variable", "control_output"], title="Observation curves")
ax.set_xlim(left=1000, right=31_000)
plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

tags=["process_variable", "control_output"]
titles = tags

for ax, tag, title in zip(axes, tags, titles):
    plot_df(ax, log_df, tags=[tag], title=title)

    if tag == "process_variable":
        ax.axhline(y=1200, color="black", linestyle="--")

plt.tight_layout()
plt.savefig(LOG_DIR / "observation_logs_split.pdf")

In [ ]:
action_tags = ["kp", "ki", "kd"]

for tag in action_tags:
    fig, ax = plt.subplots(figsize=(10, 6))

    plot_df(ax, log_df, tags=[tag], title=tag)

    ax.axhline(y=0, color="black", linestyle="--")

    plt.tight_layout()
    plt.savefig(LOG_DIR / f"{tag.replace('/', '_')}.pdf")

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 12), sharex=True)

tags=action_tags
titles = tags

for ax, tag, title in zip(axes, tags, titles):
    plot_df(ax, log_df, tags=[tag], title=title)

plt.tight_layout()
plt.savefig(LOG_DIR / "action_logs_split.pdf")

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 5))
df = log_df

x = df['step']

ax1.plot(x, df['process_variable'], label='process_variable', color='tab:blue')
ax1.plot(x, df['control_output'], label='control_output', color='tab:orange')
ax1.set_xlabel('step')
ax1.set_ylabel('PV / CO')
ax1.grid(True)
ax1.set_xlim(left=1000)

ax2 = ax1.twinx()
if 'kp' in df.columns:
    ax2.plot(x, df['kp'], label='Kp', color='tab:green')
ax2.set_ylabel('kp')

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2)
plt.title('Process variable vs Control output vs Kp')
plt.tight_layout()
plt.show()

In [ ]:
from nn_laser_stabilizer.envs.reward import (
    AbsoluteErrorReward,
    ExponentialErrorReward,
    RelativeErrorReward,
)

abs_reward = AbsoluteErrorReward()
rel_reward = RelativeErrorReward()
exp_reward_k5 = ExponentialErrorReward(k=5.0)
exp_reward_k15 = ExponentialErrorReward(k=15.0)

df = log_df[log_df["step"] >= 1000]

df['reward_absolute'] = [abs_reward(pv, sp) for pv, sp in zip(df['process_variable'], df['setpoint'])]
df['reward_relative'] = [rel_reward(pv, sp) for pv, sp in zip(df['process_variable'], df['setpoint'])]
df['reward_exponential_k5'] = [exp_reward_k5(pv, sp) for pv, sp in zip(df['process_variable'], df['setpoint'])]
df['reward_exponential_k15'] = [exp_reward_k15(pv, sp) for pv, sp in zip(df['process_variable'], df['setpoint'])]

x = df['step'] if 'step' in df.columns else range(len(df))
fig, axes = plt.subplots(2, 2, figsize=(14, 8), sharex=True)

axes[0,0].plot(x, df['reward_absolute'], color='tab:blue', linewidth=1.0)
axes[0,0].set_title('AbsoluteErrorReward')
axes[0,0].set_ylabel('reward')
axes[0,0].grid(True, alpha=0.3)

axes[0,1].plot(x, df['reward_relative'], color='tab:orange', linewidth=1.0)
axes[0,1].set_title('RelativeErrorReward')
axes[0,1].grid(True, alpha=0.3)

axes[1,0].plot(x, df['reward_exponential_k5'], color='tab:green', linewidth=1.0)
axes[1,0].set_title('ExponentialErrorReward (k=5)')
axes[1,0].set_xlabel('step')
axes[1,0].set_ylabel('reward')
axes[1,0].grid(True, alpha=0.3)

axes[1,1].plot(x, df['reward_exponential_k15'], color='tab:red', linewidth=1.0)
axes[1,1].set_title('ExponentialErrorReward (k=15)')
axes[1,1].set_xlabel('step')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()

In [ ]:
BLOCK = 2000  

df["block"] = df["step"] // BLOCK  

df_avg = (
    df.groupby("block")
    .agg({
        "reward_absolute": "mean",
        "reward_relative": "mean",
        "reward_exponential_k5": "mean",
        "reward_exponential_k15": "mean",
        "control_output": "mean",
        "process_variable": ["mean", "std"],
        "error": ["mean", "std"]
    })
    .reset_index()
)

df_avg.columns = ["_".join(col).strip("_") for col in df_avg.columns.values]

df_avg["step"] = df_avg["block"]

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8), sharex=True)

axes[0,0].plot(df_avg["step"], df_avg["reward_absolute"], color='tab:blue', linewidth=1.5)
axes[0,0].set_title('AbsoluteErrorReward (avg per 2000 steps)')
axes[0,0].set_ylabel('reward')
axes[0,0].grid(True, alpha=0.3)

axes[0,1].plot(df_avg["step"], df_avg["reward_relative"], color='tab:orange', linewidth=1.5)
axes[0,1].set_title('RelativeErrorReward (avg per 2000 steps)')
axes[0,1].grid(True, alpha=0.3)

axes[1,0].plot(df_avg["step"], df_avg["reward_exponential_k5"], color='tab:green', linewidth=1.5)
axes[1,0].set_title('ExponentialErrorReward (k=5, avg)')
axes[1,0].set_xlabel('step')
axes[1,0].set_ylabel('reward')
axes[1,0].grid(True, alpha=0.3)

axes[1,1].plot(df_avg["step"], df_avg["reward_exponential_k15"], color='tab:red', linewidth=1.5)
axes[1,1].set_title('ExponentialErrorReward (k=15, avg)')
axes[1,1].set_xlabel('step')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 8), sharex=True)

ax[0].plot(df_avg["step"], df_avg["control_output"], color='tab:blue', linewidth=1.5)
ax[0].set_title("Control output (avg per block)")
ax[0].grid(True, alpha=0.3)

ax[1].plot(df_avg["step"], df_avg["process_variable"], color='tab:green', linewidth=1.5)
ax[1].set_title("Process variable (avg per block)")
ax[1].set_xlabel("step")
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(14, 12))

ax[0].plot(df["step"], df["error"], color="tab:blue", linewidth=1)
ax[0].set_title("Error")
ax[0].set_ylabel("error")
ax[0].grid(True, alpha=0.3)

ax[1].plot(df_avg["block"], df_avg["error"], color="tab:red", linewidth=1.5)
ax[1].set_title(f"Error (усреднение по {BLOCK} шагам)")
ax[1].set_xlabel("step")
ax[1].set_ylabel("error")
ax[1].grid(True, alpha=0.3)

ax[2].plot(df_avg["block"], df_avg["error_std"], color="tab:red", linewidth=1.5)
ax[2].set_title(f"Error (усреднение по {BLOCK} шагам)")
ax[2].set_xlabel("step")
ax[2].set_ylabel("error")
ax[2].grid(True, alpha=0.3)

plt.tight_layout()